In [14]:
# Get all data from `da_expost_lmp` table for dates

from google.cloud import bigquery
from datetime import date, datetime, timedelta
import os
import pandas as pd

start_date = date(2019,1,1)
end_date = date(2019,2,1)

bq = bigquery.Client()
lmp = bq.query(
    f"""
    select *
    from grid-analysis-engine.misoenergy_org.da_expost_lmp
    where TS between TIMESTAMP('{start_date.isoformat()}', 'US/Eastern') AND TIMESTAMP('{end_date.isoformat()}', 'US/Eastern')
    order by TS
    """,
    bigquery.QueryJobConfig()
)
results = lmp.result().to_dataframe()
results

,TS,Node,LMP,MCC,MLC
0,2019-01-01 05:00:00+00:00,AMIL.GBCCTG1,17.86,-2.00,-0.40
1,2019-01-01 05:00:00+00:00,AMIL.GBCCTG2,17.86,-2.00,-0.40
2,2019-01-01 05:00:00+00:00,ALTW.MTOWNCT1,15.16,-2.00,-3.10
3,2019-01-01 05:00:00+00:00,ALTW.MTOWNCT2,15.16,-2.00,-3.10
4,2019-01-01 05:00:00+00:00,EES.SABINECO,22.99,2.25,0.48
...,...,...,...,...,...
1597729,2019-02-01 05:00:00+00:00,AMIL.CSTNCTG1,34.57,0.00,-1.18
1597730,2019-02-01 05:00:00+00:00,AMIL.CSTNCTG2,34.57,0.00,-1.18
1597731,2019-02-01 05:00:00+00:00,ALTW.WPPI_1.AZ,35.36,0.00,-0.39
1597732,2019-02-01 05:00:00+00:00,MEC.MEAN_10.AZ,36.28,0.00,0.53


In [16]:
# Pivot tables to be by `Node` for `LMP` values

results.pivot_table(index=['TS'], columns='Node', values='LMP')

Node,AEC,AECI,AECI.ALTW,AECI.AMMO,AECI.APM_1.AZ,AECI.APM_2.AZ,AECI.CWLD,AEP,AEP.PSGC1.AMP,AEP.PSGC2.AMP,...,WPS.WPSM,WR,WR.MEUCFRED1,WR.MEUCFRED2,WR.MEUCLADD,WR.MEUC_1.AZ,WR.MOWR,WR.MOWR_1.AZ,WR.THAYERLD,YAD
TS,,,,,,,,,,,,,,,,,,,,,
2019-01-01 05:00:00+00:00,20.77,17.61,18.77,19.32,19.23,18.30,18.30,20.37,18.96,18.96,...,21.17,15.05,19.85,19.85,19.31,19.81,19.65,19.59,19.81,20.83
2019-01-01 06:00:00+00:00,20.33,17.45,18.48,19.29,19.16,18.01,18.01,20.05,18.68,18.70,...,20.90,14.68,19.73,19.73,18.91,19.97,19.53,19.50,19.97,20.39
2019-01-01 07:00:00+00:00,20.39,17.40,18.42,19.03,18.91,18.10,18.10,19.99,18.70,18.70,...,20.93,14.96,19.40,19.40,18.90,19.32,19.26,19.21,19.32,20.25
2019-01-01 08:00:00+00:00,20.25,17.42,18.95,19.25,19.13,18.07,18.07,19.96,18.86,18.85,...,20.78,14.98,19.71,19.71,18.91,19.50,19.58,19.54,19.50,20.31
2019-01-01 09:00:00+00:00,20.54,18.10,18.94,19.17,19.05,18.23,18.23,20.10,18.84,18.83,...,21.00,16.30,19.54,19.54,19.04,20.09,19.40,19.35,20.09,20.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-02-01 01:00:00+00:00,24.72,46.11,121.50,64.18,62.32,50.53,50.53,95.87,64.30,64.32,...,104.42,53.32,54.21,54.21,68.29,43.43,54.90,56.43,43.43,97.92
2019-02-01 02:00:00+00:00,23.92,45.00,103.87,57.92,56.39,47.64,47.64,83.91,57.82,57.84,...,89.90,45.29,49.38,49.38,69.08,47.08,50.15,51.52,47.08,85.95
2019-02-01 03:00:00+00:00,29.82,34.94,102.95,47.73,47.09,41.98,41.98,53.45,46.83,46.83,...,67.08,41.97,48.34,48.34,49.13,47.22,47.20,46.86,47.22,56.58
